In [ ]:
#| all_slow

In [ ]:
!pip install datasets --upgrade
!pip install lightgbm --upgrade

In [ ]:
from relax.import_essentials import *
from relax.data import *
from relax.module import *
import datasets as hfds
from relax.trainer import train_model, TrainingConfigs
from relax.utils import *
from relax._ckpt_manager import load_checkpoint, save_checkpoint

In [ ]:
ds = hfds.load_dataset("birkhoffg/folktables-acs-income")

Found cached dataset parquet (/home/birk/.cache/huggingface/datasets/birkhoffg___parquet/birkhoffg--folktables-acs-income-bc190711a423bf3e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def hfds_to_dm(
    dataset: hfds.Dataset, 
    configs: TabularDataModuleConfigs
) -> TabularDataModule:
    train_df = dataset["train"].to_pandas()
    test_df = dataset["test"].to_pandas()
    df = pd.concat([train_df, test_df])
    if "__index_level_0__" in df.columns:
        df = df.drop(columns=["__index_level_0__"])
    print('df is loaded')
    dm = TabularDataModule(configs, df)
    return dm

In [ ]:
configs = TabularDataModuleConfigs(
    data_dir='',
    data_name='forktable',
    continous_cols=['AGEP', 'OCCP', 'POBP', 'RELP', 'WKHP'],
    discret_cols=['COW', 'SCHL', 'MAR', 'SEX', 'RAC1P', 'STATE', 'YEAR'],
    # sample_frac=0.1
)

In [ ]:
dm = hfds_to_dm(ds, configs)

df is loaded


/home/birk/mambaforge-pypy3/envs/nbdev2/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
module = PredictiveTrainingModule({
    'lr': 1e-3,
    'sizes': [110, 110, 50, 10],
    'dropout': 0.3,
})

In [ ]:
params, _ = train_model(
    module, dm, TrainingConfigs(
        n_epochs=10, batch_size=256, monitor_metrics='val/val_accuracy',
        max_n_checkpoints=1
    )
)

Epoch 9: 100%|██████████| 28694/28694 [02:57<00:00, 161.59batch/s, train/train_loss_1=0.0623]


In [ ]:
train_X, train_y = dm.train_dataset[:]
test_X, test_y = dm.test_dataset[:]

In [ ]:
y_pred = module.pred_fn(test_X, params, jrand.PRNGKey(0)).round()
(y_pred == test_y).mean()

In [ ]:
import lightgbm as lgb

In [ ]:
clf = lgb.LGBMClassifier(
    n_estimators=100,
)
clf.fit(train_X, train_y)

/home/birk/mambaforge-pypy3/envs/nbdev2/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/birk/mambaforge-pypy3/envs/nbdev2/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


LGBMClassifier()